# Logistic Regression on MNIST Data

Run each cell in the notebook. The explanation of the cells are given on top respectively. Also, you need to have keras with tensorflow backend to load the dataset.

In [320]:
import numpy as np
import random
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras_team.keras.datasets import mnist

Downloading the Mnist data

In [321]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Scaling the data so that values are between 0 and 1

In [322]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
x_train = x_train/255
x_test = x_test/255

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


Creating the training and testing dataset alongwith setting the Y values to 0 or 1 depending on a condition. The condition here is last digit of VUNetId. My last digit is 1. So for the Y values of 1, new value is set to 1(true). Rest of them are set to 0(false). Also the 28x28 dataset is converted to a column vector 784x1.

In [323]:
X = x_train.reshape(60000, 784)  #Converting to column vector for each datapoint
X1 = X
Y = np.array(y_train == 1)
Y = 1*Y    #Setting values to 1 or 0 for training

test_X = x_test.reshape(10000,784)
test_X = test_X.transpose()
test_Y = np.array(y_test == 1)
test_Y = 1*test_Y         # Setting values to 1 or 0 for test


This is the forward propagation step. Here W, b and the data is received as input. The operation W.X + b is performed after taking appropriate transpose. The final value is then passed through a sigmoid function to make the value between 0 - 1

In [324]:
def forward_prop(W, b, data_X):
    Wt = W.transpose()
    intermediate = np.inner(Wt, data_X.transpose())
    Z = intermediate + b
    A = sigmoid(Z)
    return A

This is the backward propagation step. Here the required derivatives are calculated which will be later updated in further steps

In [325]:
def backward_prop(A, m, data_X, data_Y):
    dZ = A - data_Y
    dW = np.inner(data_X, dZ)/m
    db = sum(dZ)/m
    return dW, db

This is the sigmoid activation function used to map any real value between 0 and 1

In [326]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Computes the cost function for all the training samples

In [327]:
def cost_function(A, m, data_Y):
    total_cost = -(1 / m) * np.sum(data_Y * np.log(A) + (1 - data_Y) * np.log(1 - A))
    return total_cost

Here the results are predicted. The values are predicted using test dataset and then using the actual values the accuracy is computed

In [328]:
def results(W, b, data_X, data_Y):
    Wt = W.transpose()
    predicted = np.inner(Wt, data_X.transpose())
    predicted = predicted + b
    pred_Y = sigmoid(predicted)
    pred_Y = pred_Y.transpose()
    pred_Y = np.around(pred_Y, decimals = 0)   #Rounding off the predicted value to 0 or 1
    accuracy = accuracy_score(data_Y, pred_Y)
    return accuracy

This is the logistic regression function. First we split the training data into training and validation sets. Then we initialize W and b with random numbers. After that using W, b and the training data we implement the forward propagation. Using the results we calculate the derivatives for the backward propagation step. After that we update our b and W using the derivative values obtained. We then test this using the validation data. We store the alpha for which accuracy was the highest.

In [329]:
def logistic_reg():    
    alpha = [0.0001, 0.001, 0.005, 0.01, 0.1, 0.05]   #set of learning rates to check from
    epochs = 1001
    max_cost = 0
    max_accuracy = 0
    alpha_max = 0
    for i in range(len(alpha)):
        train_X, valid_X, train_Y, valid_Y = train_test_split(X, Y, test_size=0.2) # Splitting training data using 80:20 rule
        valid_X = valid_X.transpose()
        train_X = train_X.transpose()
        n, m = train_X.shape
        W = np.random.rand(n,1)     #Initializing W and b
        b = random.random()
        for j in range(epochs):
            if(j%500 == 0):
                print('Epoch:', j)
            A = forward_prop(W, b, train_X)
            dW, db = backward_prop(A, m, train_X, train_Y)
            W = W - alpha[i]*dW                    #Updating b and W
            b = b - alpha[i]*db
        b = sum(b)/m                  
        accuracy = results(W, b, valid_X, valid_Y)   #testing the accuracy using validation set
        if(accuracy >= max_accuracy):
            alpha_max = alpha[i]                     #finding the alpha for which accuracy was the maximum
            max_accuracy = accuracy
        print('Alpha:',alpha[i],'Accuracy:', accuracy)
    print( 'Max:',alpha_max)
    return alpha_max
alpha_max = logistic_reg()

Epoch: 0
Epoch: 500
Epoch: 1000
Alpha: 0.0001 Accuracy: 0.115
Epoch: 0
Epoch: 500
Epoch: 1000
Alpha: 0.001 Accuracy: 0.11258333333333333
Epoch: 0
Epoch: 500
Epoch: 1000
Alpha: 0.005 Accuracy: 0.84825
Epoch: 0
Epoch: 500
Epoch: 1000
Alpha: 0.01 Accuracy: 0.9405
Epoch: 0
Epoch: 500
Epoch: 1000
Alpha: 0.1 Accuracy: 0.9813333333333333
Epoch: 0
Epoch: 500
Epoch: 1000
Alpha: 0.05 Accuracy: 0.97725
Max: 0.1


Here, we calculate the cost function for each iteration using the learning rate which we had obtained earlier. We run this on the entire training set. After obatining values of W and b, we calculate the training and test error.

In [ ]:
X = X.transpose()
n, m = X.shape
W = np.random.rand(n,1)
b = random.random()
epochs = 2001
cost_array = []
J_arr =[]
count = 0
for j in range(epochs):
    if(j%500 == 0):
        print('Epoch:', j)
    A = forward_prop(W, b, X)
    dW, db = backward_prop(A, m, X, Y)
    W = W - alpha_max*dW
    b = b - alpha_max*db
    total_cost = cost_function(A, m, Y)
    cost_array.append(total_cost)    # Storing the value of cost function
    J_arr.append(count)
    count+= 1
b = sum(b)/m
final_accuracy_test = results(W, b, test_X, test_Y)*100    # Calculating the training and test accuracy
final_accuracy_train = results(W, b, X1.transpose(), Y)*100
print('Training error:', str(100-final_accuracy_train),'%')
print('Test error:', str(100-final_accuracy_test),'%')

Epoch: 0


C:\Users\Bikram Kumar De\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
C:\Users\Bikram Kumar De\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in multiply
  


Epoch: 500
Epoch: 1000


Plotting the cost function as a function of the iterations. We can see that the cost function decreases very fast initially but after a certain number of iterations the rate of decrease is much less which indicates that our logistic regression is  converging hence the error is reducing asymptotically.

In [ ]:
plt.title('Cost Function Graph')
plt.xlabel('Epoch')
plt.ylabel('Cost Function')
plt.plot(J_arr, cost_array)
plt.show()

Here, you can check any individual image. You can load the particular image from the test dataset by changing the array index. The program will tell whether the number is 1 or not

In [ ]:
x1 = x_test[2]
plt.imshow(x1, interpolation='nearest', cmap = 'gray')
plt.show()
x1 = x1.reshape(784, 1)
A = forward_prop(W, b, x1)
if (A<0.5):
    print('This is not 1')
else:
    print('This is 1')


In [ ]:
x1 = x_test[2566]
plt.imshow(x1, interpolation='nearest', cmap = 'gray')
plt.show()
x1 = x1.reshape(784, 1)
A = forward_prop(W, b, x1)
if (A<0.5):
    print('This is not 1')
else:
    print('This is 1')

Higher learning rate means the regression learns faster, i.e. it converges faster. The accuracy at end of 1000 epochs may not show significant changes on changing the learning rate because in this small dataset, having smaller alphas also makes the program converge. 